In [1]:
from bs4 import BeautifulSoup
import requests
import datetime
import pandas
import tqdm
import re

In [2]:
def get_date(num):
    days = []
    for i in range(num)[1:]:
        days.append( str((datetime.date.today()-datetime.timedelta(days=i))).replace('-','.') )
    return days

In [3]:
def get_titles(data):
    r = requests.get('http://www.vesti.ru/news/index/date/'+'2019.05.19') #'2019.05.19' формат
    soup = BeautifulSoup(r.content)
    title = [(i.a['href'], i.a.text) for i in soup.find_all('h3', {"class":"b-item__title"})]
    return title

In [4]:
def get_text(url):
    try:
        rq = requests.get('https://vesti.ru'+url)
        soup_new = BeautifulSoup(rq.content)
        text = ''.join([ " ".join(re.split("\s+", i.text.replace('\t', ' ').replace('\n', ' ').replace('\xa0', ' '),flags=re.UNICODE))  for i in soup_new.find_all('p')[:-1]])
        return text
    except:
        try:
            rq = requests.get(url)
            soup_new = BeautifulSoup(rq.content)
            text = ''.join([ " ".join(re.split("\s+", i.text.replace('\t', ' ').replace('\n', ' ').replace('\xa0', ' '),flags=re.UNICODE))  for i in soup_new.find_all('p')[:-1]])
            return text
        except:
            print('error')
            return 'None'

In [7]:
def writer(name,num):
    df = pandas.DataFrame(columns=["date","url","title","text"])
    for date in tqdm.tqdm_notebook(get_date(num)):
        for i in tqdm.tqdm_notebook(get_titles(date)):
            txt = get_text(i[0])
            dct = {"date": date, 
                   "url": 'https://vesti.ru'+i[0],
                   "title": i[1],
                   "text": txt}
            df = df.append(dct, ignore_index=True)
    df.to_csv(name)
    return df

In [6]:
df = writer('vesti.csv',30)